In [ ]:
import json,datetime,pymysql
import os
import csv
from datetime import datetime

conn = pymysql.connect(host='mysql.clarksonmsda.org', port=3306, user='gonuguc',
                       passwd='*******', db='gonuguc_Weather', autocommit=True)
cur = conn.cursor(pymysql.cursors.DictCursor)

sql = '''DROP TABLE IF EXISTS Observation_data
'''

cur.execute(sql)

#creating a new table
sql = """
CREATE TABLE Observation_data (
    Weather_id int NOT NULL AUTO_INCREMENT,
    Station_id VARCHAR(255),
    DATE TIMESTAMP,
    TIME VARCHAR(255),
    QUALITY VARCHAR(255),
    WIND VARCHAR(255),
    CEILING  VARCHAR(255),
    VISIBILITY  VARCHAR(255),
    TEMPERATURE  VARCHAR(255),
    DEW_POINT  VARCHAR(255),
    SEA_LEVEL_PRESSURE  VARCHAR(255),
    PRIMARY KEY(Weather_id)
    )ENGINE=MyISAM DEFAULT CHARSET=utf8mb4 AUTO_INCREMENT=1 COLLATE=utf8mb4_0900_ai_ci;
"""
cur.execute(sql)
n=0
#function to iterate files in folder
def list_files_in_folder(folder_path):
    try:
        files = os.listdir(folder_path)
        files = [file for file in files if os.path.isfile(os.path.join(folder_path, file))]
        return files
    except Exception as e:
        print(f"Error listing files: {e}")
        return []
folder_path = "/Users/avinashkalamati/Desktop/IA626/project/2022/"#folder location 
files_in_folder = list_files_in_folder(folder_path)
if files_in_folder:
    print(f"Files in folder '{folder_path}':")
    for file in files_in_folder:
        n+=1
        print(n)
        if n>500:
            # exit after 500 stations
            break
        else:
            with open (file) as f:
                data = [{k: str(v) for k, v in row.items()}
                for row in csv.DictReader(f, skipinitialspace=True)]
                conn = pymysql.connect(host='mysql.clarksonmsda.org', port=3306, user='gonuguc',
                                passwd='Chandu@1604', db='gonuguc_Weather', autocommit=True)
                cur = conn.cursor(pymysql.cursors.DictCursor)
                    
                try:
                    sql = '''INSERT INTO Observation_data (Station_id,DATE,TIME,QUALITY,WIND,CEILING,VISIBILITY,TEMPERATURE,DEW_POINT,SEA_LEVEL_PRESSURE) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
                    blocksize = 10000
                    tokens = []
                    for row in data:
                        #date
                        date = datetime.strptime(row['DATE'],'%Y-%m-%dT%H:%M:%S').strftime("%Y-%m-%d")
                        #time
                        time = datetime.strptime(row['DATE'],'%Y-%m-%dT%H:%M:%S').strftime("%H:%M:%S")
                        #wind
                        string = row['WND']
                        if string.split(',')[0][-3:] == '999':
                            wind=None
                        else:
                            wind=str(string.split(',')[0])+str(string.split(',')[2])                        
                            #ceiling
                            string = row['CIG']
                            if string.split(',')[0][-3:] == '999':
                                ceil = None
                            else:
                                ceil = int(string.split(',')[0])
                                
                            #visibility
                            string = row['VIS']
                            if string.split(',')[0][-3:] == '999':
                                visi = None
                            else:
                                visi = int(string.split(',')[0])
                                
                            #Temperature
                            string = row['TMP']
                            if string.split(',')[0][-3:] == '999':
                                temp=None
                            else:
                                temp=int(string.split(',')[0])/10

                            #DEW point
                            string = row['DEW']
                            if string.split(',')[0][-3:] == '999':
                                dew_point=None
                            else:
                                dew_point=int(string.split(',')[0])/10
                                
                            #SEA LEVEL PRESSURE
                            string = row['SLP']
                            if string.split(',')[0][-3:] == '999':
                                sea=None
                            else:
                                sea = int(string.split(',')[0])/10
                            tokens.append([row['STATION'],date,time,row['QUALITY_CONTROL'],wind,ceil,visi,temp,dew_point,sea])
                    if len(tokens) >= blocksize:
                        cur.executemany(sql, tokens)#sending data as blocks of size 10000
                        conn.commit()
                except Exception as e:
                    print(e)        
cur.close()
conn.close()